In [9]:
import pandas as pd
import numpy as np
import sklearn

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [11]:
%pip install uv



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
uv --version


uv 0.9.5
Note: you may need to restart the kernel to use updated packages.


In [13]:
!python3 --version


Python 3.12.1


In [15]:
import os
print(os.getcwd())


/workspaces/DataTalksMlZoomCamp/05-Deployment


In [17]:
import pickle
import pandas as pd

# load the pipeline
with open("/workspaces/DataTalksMlZoomCamp/homework/pipeline_v1.bin", "rb") as f:
    pipeline = pickle.load(f)

# record to score
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

# use a DataFrame in case pipeline expects it
X = [record]

# get probability of conversion (usually positive class = 1)
proba = pipeline.predict_proba(X)[0, 1]

print("Probability of conversion:", proba)


Probability of conversion: 0.5336072702798061


In [1]:
!pip install fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [fastapi]5;237m━━━━ 8/9 [fastapi]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [8]:
pip install scikit-learn==1.6.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 61.4 MB/s  0:00:00s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.0
    Uninstalling scikit-learn-1.7.0:
      Successfully uninstalled scikit-learn-1.7.0

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pickle
from typing import Literal
from pydantic import BaseModel, Field
from fastapi import FastAPI
import uvicorn


# -----------------------------
# Input schema
# -----------------------------
class Client(BaseModel):
    lead_source: Literal["paid_ads", "organic_search", "referral", "other"]
    number_of_courses_viewed: int = Field(..., ge=0)
    annual_income: float = Field(..., ge=0.0)


# -----------------------------
# Response schema
# -----------------------------
class PredictResponse(BaseModel):
    subscription_probability: float
    subscribed: bool


import os
print("Current working dir:", os.getcwd())
print("Files:", os.listdir("."))


# -----------------------------
# FastAPI app
# -----------------------------
app = FastAPI(title="subscription-prediction")

with open("homework/pipeline_v1.bin", "rb") as f_in:
    pipeline = pickle.load(f_in)


def predict_single(client):
    X = [client]
    result = pipeline.predict_proba(X)[0, 1]
    return float(result)


@app.post("/predict", response_model=PredictResponse)
def predict(client: Client) -> PredictResponse:
    prob = predict_single(client.model_dump())

    return PredictResponse(
        subscription_probability=prob,
        subscribed=prob >= 0.5
    )


# -----------------------------
# Run with uvicorn
# -----------------------------
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. Th

Current working dir: /workspaces/DataTalksMlZoomCamp/05-Deployment
Files: ['homework', 'HomeWork.ipynb', '.ipynb_checkpoints']


RuntimeError: asyncio.run() cannot be called from a running event loop

In [12]:
import requests

url = "http://0.0.0.0:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())


{'subscription_probability': 0.5340417283801275, 'subscribed': True}
